## 1. Classic American names
<p><img src="https://assets.datacamp.com/production/project_1441/img/name.jpg" alt="Lots of name tags piled on top of each other." width="600"></p>
<p>Photo by Travis Wise on <a href="https://commons.wikimedia.org/wiki/File:Hello_My_Name_Is_(15283079263).jpg">Wikimedia</a>.</p>
<p>How have American baby name tastes changed since 1920?  Which names have remained popular for over 100 years, and how do those names compare to more recent top baby names? These are considerations for many new parents, but the skills we'll practice while answering these queries are broadly applicable. After all, understanding trends and popularity is important for many businesses, too! </p>
<p>We'll be working with data provided by the United States Social Security Administration, which lists first names along with the number and sex of babies they were given to in each year. For processing speed purposes, we've limited the dataset to first names which were given to over 5,000 American babies in a given year. Our data spans 101 years, from 1920 through 2020.</p>
<h3 id="baby_names"><code>baby_names</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>year</td>
</tr>
<tr>
<td style="text-align:left;"><code>first_name</code></td>
<td>varchar</td>
<td>first name</td>
</tr>
<tr>
<td style="text-align:left;"><code>sex</code></td>
<td>varchar</td>
<td><code>sex</code> of babies given <code>first_name</code></td>
</tr>
<tr>
<td style="text-align:left;"><code>num</code></td>
<td>int</td>
<td>number of babies of <code>sex</code> given <code>first_name</code> in that <code>year</code></td>
</tr>
</tbody>
</table>
<p>Let's get oriented to American baby name tastes by looking at the names that have stood the test of time!</p>

In [2]:
%%sql
SELECT first_name, SUM(num) AS sum
FROM baby_names
GROUP BY first_name
HAVING COUNT(DISTINCT year) = 101
ORDER BY sum DESC;

,first_name,sum
0,James,4748138
1,John,4510721
2,William,3614424
3,David,3571498
4,Joseph,2361382
5,Thomas,2166802
6,Charles,2112352
7,Elizabeth,1436286


## 2. Timeless or trendy?
<p>Wow, it looks like there are a lot of timeless traditionally male names! Elizabeth is holding her own for the female names, too. </p>
<p>Now, let's broaden our understanding of the dataset by looking at all names. We'll attempt to capture the type of popularity that each name in the dataset enjoyed. Was the name classic and popular across many years or trendy, only popular for a few years? Let's find out. </p>

In [3]:
%%sql
WITH NamePopularity AS (
    SELECT 
        first_name, 
        SUM(num) AS sum,
        COUNT(DISTINCT year) AS years_present
    FROM 
        baby_names
    GROUP BY 
        first_name
)
SELECT 
    first_name,
    sum,
    CASE 
        WHEN years_present > 80 THEN 'Classic'
        WHEN years_present > 50 THEN 'Semi-classic'
        WHEN years_present > 20 THEN 'Semi-trendy'
        ELSE 'Trendy'
    END AS popularity_type
FROM 
    NamePopularity
ORDER BY 
    first_name;

,first_name,sum,popularity_type
0,Aaliyah,15870,Trendy
1,Aaron,530592,Semi-classic
2,Abigail,338485,Semi-trendy
3,Adam,497293,Semi-trendy
4,Addison,107433,Trendy
...,...,...,...
542,Wyatt,128168,Trendy
543,Xavier,51892,Trendy
544,Zachary,483955,Semi-trendy
545,Zoe,78773,Trendy


## 3. Top-ranked female names since 1920
<p>It's time to dive into the female names! </p>
<p>Since we didn't get many traditionally female names in our classic American names search in the first task, let's limit our search to names which were given to female babies. </p>
<p>We can use this opportunity to practice window functions by assigning a rank to female names based on the number of babies that have ever been given that name. What are the top-ranked female names since 1920?</p>

In [4]:
%%sql
WITH FemaleNameRank AS (
    SELECT 
        first_name,
        SUM(num) AS sum,
        RANK() OVER (ORDER BY SUM(num) DESC) AS name_rank
    FROM 
        baby_names
    WHERE 
        sex = 'F'
    GROUP BY 
        first_name
)
SELECT 
    name_rank,
    first_name,
    sum
FROM 
    FemaleNameRank
LIMIT 10;

,name_rank,first_name,sum
0,1,Mary,3215850
1,2,Patricia,1479802
2,3,Elizabeth,1436286
3,4,Jennifer,1404743
4,5,Linda,1361021
5,6,Barbara,1343901
6,7,Susan,1025728
7,8,Jessica,994210
8,9,Lisa,920119
9,10,Betty,893396


## 4. Picking a baby name
<p>Perhaps a friend has heard of our work analyzing baby names and would like help choosing a name for her baby, a girl. She doesn't like any of the top-ranked names we found in the previous task. </p>
<p>She's set on a traditionally female name ending in the letter 'a' since she's heard that vowels in baby names are trendy. She's also looking for a name that has been popular in the years since 2015. </p>
<p>Let's see what we can do to find some options for this friend!</p>

In [5]:
%%sql
SELECT 
    first_name
FROM 
    baby_names
WHERE 
    sex = 'F'
    AND year > 2015
    AND first_name LIKE '%a'
GROUP BY 
    first_name
ORDER BY 
    SUM(num) DESC;

,first_name
0,Olivia
1,Emma
2,Ava
3,Sophia
4,Isabella
5,Mia
6,Amelia
7,Ella
8,Sofia
9,Camila


## 5. The Olivia expansion
<p>Based on the results in the previous task, we can see that Olivia is the most popular female name ending in 'A' since 2015. When did the name Olivia become so popular?</p>
<p>Let's explore the rise of the name Olivia with the help of a window function.</p>

In [6]:
%%sql
SELECT 
    year,
    first_name,
    num,
    SUM(num) OVER (ORDER BY year) AS cumulative_olivias
FROM 
    baby_names
WHERE 
    first_name = 'Olivia'
ORDER BY 
    year;

,year,first_name,num,cumulative_olivias
0,1991,Olivia,5601,5601
1,1992,Olivia,5809,11410
2,1993,Olivia,6340,17750
3,1994,Olivia,6434,24184
4,1995,Olivia,7624,31808
5,1996,Olivia,8124,39932
6,1997,Olivia,9477,49409
7,1998,Olivia,10610,60019
8,1999,Olivia,11255,71274
9,2000,Olivia,12852,84126


## 6. Many males with the same name
<p>Wow, Olivia has had a meteoric rise! Let's take a look at traditionally male names now. We saw in the first task that there are nine traditionally male names given to at least 5,000 babies every single year in our 101-year dataset! Those names are classics, but showing up in the dataset every year doesn't necessarily mean that the timeless names were the most popular. Let's explore popular male names a little further.</p>
<p>In the next two tasks, we will build up to listing every year along with the most popular male name in that year. This presents a common problem: how do we find the greatest X in a group? Or, in the context of this problem, how do we find the male name given to the highest number of babies in a year? </p>
<p>In SQL, one approach is to use a subquery. We can first write a query that selects the <code>year</code> and the maximum <code>num</code> of babies given any single male name in that year. For example, in 1989, the male name given to the highest number of babies was given to 65,339 babies. We'll write this query in this task. In the next task, we can use the code from this task as a subquery to look up the <code>first_name</code> that was given to 65,339 babies in 1989… as well as the top male first name for all other years!</p>

In [7]:
%%sql
SELECT 
    year,
    MAX(num) AS max_num
FROM 
    baby_names
WHERE 
    sex = 'M'
GROUP BY 
    year;

,year,max_num
0,1964,82642
1,1969,85201
2,2008,22603
3,1991,60793
4,1989,65399
...,...,...
96,1923,57469
97,2012,19088
98,1954,88576
99,1988,64150


## 7. Top male names over the years
<p>In the previous task, we found the maximum number of babies given any one male name in each year. Incredibly, the most popular name each year varied from being given to less than 20,000 babies to being given to more than 90,000! </p>
<p>In this task, we find out what that top male name is for each year in our dataset. </p>

In [8]:
%%sql
SELECT 
    subquery.year,
    baby_names.first_name,
    subquery.max_num AS num
FROM 
    baby_names
INNER JOIN (
    SELECT 
        year,
        MAX(num) AS max_num
    FROM 
        baby_names
    WHERE 
        sex = 'M'
    GROUP BY 
        year
) AS subquery
ON 
    baby_names.year = subquery.year
    AND baby_names.num = subquery.max_num
    AND baby_names.sex = 'M'
ORDER BY 
    subquery.year DESC;

,year,first_name,num
0,2020,Liam,19659
1,2019,Liam,20555
2,2018,Liam,19924
3,2017,Liam,18824
4,2016,Noah,19154
...,...,...,...
96,1924,Robert,60801
97,1923,John,57469
98,1922,John,57280
99,1921,John,58215


## 8. The most years at number one
<p>Noah and Liam have ruled the roost in the last few years, but if we scroll down in the results, it looks like Michael and Jacob have also spent a good number of years as the top name! Which name has been number one for the largest number of years? Let's use a common table expression to find out. </p>

In [9]:
%%sql
WITH TopNames AS (
    SELECT 
        subquery.year,
        baby_names.first_name,
        subquery.max_num AS num_babies
    FROM 
        baby_names
    INNER JOIN (
        SELECT 
            year,
            MAX(num) AS max_num
        FROM 
            baby_names
        WHERE 
            sex = 'M'
        GROUP BY 
            year
    ) AS subquery
    ON 
        baby_names.year = subquery.year
        AND baby_names.num = subquery.max_num
        AND baby_names.sex = 'M'
)
SELECT 
    first_name,
    COUNT(*) AS count_top_name
FROM 
    TopNames
GROUP BY 
    first_name
ORDER BY 
    count_top_name DESC;

,first_name,count_top_name
0,Michael,44
1,Robert,17
2,Jacob,14
3,James,13
4,Liam,4
5,John,4
6,Noah,4
7,David,1
